Membuat variasi word 2 vector dengan source textrank pke, yg bisa mengolah phrase N-gram

In [1]:
#1. rutin1 import module
import pandas as pd
import os
import sys
from sklearn.metrics.pairwise import cosine_similarity

In [2]:
#2. rutin2 membuat syspath ke root utk aktifkan __init__.py
repo_root = os.path.abspath(os.path.join(os.getcwd(), os.pardir, os.pardir))
sys.path.append(repo_root)

In [4]:
#3. rutin3 Load the dataset
dataset_path = os.path.join(repo_root, "notebooks/postager_nlp-id/dataset_ekstraksi_r29_pos_sm.xlsx")
df = pd.read_excel(dataset_path)
df_pos = df['pos_sentence_list']

In [5]:
# import module dalam path utils?? belum berhasil
from utils import TextRank
from utils import LoadFile

textrank = TextRank()

pos = {'NN', 'VP', 'NP', 'FW'}
# 1. create a TextRank extractor.
extractor = textrank.TextRank()

ImportError: cannot import name 'LoadFile' from 'utils' (/Users/jim/Documents/GitHub/kw_ina_extraction/utils/__init__.py)

In [ ]:
import ast

df_keyphrases = pd.DataFrame(columns=['key_1', 'val_1', 'key_2', 'val_2', 'key_3', 'val_3'])

for text in df_pos:
    text1 = ast.literal_eval(text)
    extractor.load_document(input=text1) # text is not a list of list of tuple
    keyphrase = extractor.candidate_weighting_ia()

        # Separate the key and value elements into separate lists
    keys = [item[0] for item in keyphrase]
    values = [item[1] for item in keyphrase]

    # Create a dictionary with the keys and values
    data = {
    'key_1': [keys[0]],
    'val_1': [values[0]],
    'key_2': [keys[1]],
    'val_2': [values[1]],
    'key_3': [keys[2]],
    'val_3': [values[2]],
    }
    df_keyphrase = pd.DataFrame(data)
    
    # Concatenate df_keyphrase with df_keyphrases
    df_keyphrases = pd.concat([df_keyphrases, df_keyphrase], ignore_index=True)

In [ ]:
# Add df_keyphrases ke file excel dataset_ekstraksi_r30.xlsx
result_df = pd.concat([df, df_keyphrases], axis=1)

# simpan dalam dataset
output_path = os.path.join(PROJECT_ROOT, "data", "dataset_ekstraksi_r30.xlsx")
result_df.to_excel(output_path, index=False)

In [ ]:
# Write predictions to excel file
from utils import write_excel

sheet_name = 'w2v_pke_textrank'
output_file = 'w2v_pke_textrank.xlsx'
write_excel(df_keyphrases, sheet_name, output_file)